In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
import chromadb
import ollama
import os
from tqdm import tqdm
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings

In [2]:
path=r"D:/HRAgent/Policies/ANTI-BRIBERY AND ANTI-CORRUPTION POLICY OF TATA INDUSTRIES.pdf"

In [3]:
loader=PyPDFLoader(path)

In [4]:
resultantpdf=[]

In [5]:
resultantpdf.extend(loader.load())

In [6]:
path=r"D:/HRAgent/Policies"
fname=os.listdir(path)
resultantpdf=[]
for fl in fname:
    
    loader=PyPDFLoader(os.path.join(path,fl))
    resultantpdf.extend(loader.load())

In [7]:
url="http://172.16.0.178:8010"

In [8]:
# embedding=OllamaEmbeddings(model="llama3.2:1b",base_url=url)
embedding = SpacyEmbeddings(model_name="en_core_web_sm")

In [9]:
semanticchunker=SemanticChunker(embedding, breakpoint_threshold_type="percentile")

In [10]:
chunks=semanticchunker.create_documents([doc.page_content for doc in tqdm(resultantpdf)])

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


In [11]:
modified_chunks=[]
for ch in chunks:
    if ch.page_content=="":
        pass
    else:
        modified_chunks.append(ch)
    

In [12]:
len(modified_chunks)

129

In [ ]:
chroma_db = Chroma.from_documents(
    documents=modified_chunks, 
    embedding=embedding, 
    persist_directory="knowledgebase3", 
    collection_name="spacy_embed_knolwdge"
)

In [11]:
chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")

C:\Users\AdityaKumarSingh\AppData\Local\Temp\ipykernel_3252\4032791689.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")


[Document(metadata={}, page_content='Expenses must never be hidden or purposefully misclassified. Many serious global \nbribery and corruption scenarios are found to invo lve inaccurate record -keeping. To \nprevent this, international anti-corruption laws generally require detailed and accurate \naccounting records for transactions, inc luding cash and bank accounts. All \nDesignated Persons must ensure that we maintain accurate books, records and \nfinancial reporting. 3. All business units and entities must maintain an effective system of internal control \nand m onitoring of our transactions. Certain monitoring controls are identified in our \npolicies, specifically regarding approval of travel and entertainme nt expenses. It is the \nresponsibility of the Designated Persons to be knowle dgeable about control \nprocedures and ensure compliance. Designated Persons are required to ensure that \nall expense claims relating to hospitality, gifts or charitable donations are submitted in

In [12]:
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you do not know the answer, please think rationally and answer from your own knowledge base.
Don't put extra information

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [13]:
llm=ChatOllama(base_url=url,model="llama3.2:1b",temperature=0.1)

In [14]:
query="finance vp of tata iq?"

In [15]:
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=chroma_db.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs)

qa_chain.run(query)

C:\Users\AdityaKumarSingh\AppData\Local\Temp\ipykernel_3252\2838688471.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


"A gift is considered acceptable if it meets certain criteria, as outlined in Section D, Clause 11 and Section G, Clause 4 of the Tata Consultancy Company's (TCoC) 2015 Gifts & Hospitality Policy. These criteria include:\n\n- Reasonableness: The gift should be proportionate to the relationship between the parties involved.\n- Proportionality: The gift should not be excessive or unreasonable in terms of value or frequency.\n- Good faith: The gift should be made in good faith and without any ulterior motive.\n- Compliance with policies: The gift should comply with the Company's Gifts & Hospitality Policy, including Section D, Clause 11 and Section G, Clause 4.\n\nIn the context of a finance VP at Tata IQ, a gift would likely be considered acceptable if it meets these criteria."

In [ ]:
from pydantic import BaseModel
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

In [ ]:
llm=ChatOllama(base_url="http://172.16.0.178:8010/",model="llama3.2:1b")

In [ ]:
class GradeDocument(BaseModel):
    binary_score: str= Field("Give a binary score 'yes' or 'no' score to indicate whether the document is relevant or not")

structure_llm_grader= llm.with_structured_output(GradeDocument)

In [ ]:
structure_llm_grader

In [ ]:
system=""" You are a grader assessing relevance of a retrieved document to a user question.\n
If the document contains keyword(s) or semantic meaning related to the question, grade it as a relevent.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant or not

"""

In [ ]:
grade_prompt= ChatPromptTemplate.from_messages([
    ("system",system),
    ("human","""Retrieved documents: \n\n {documents} \n\n user_question: {question}""")
])

In [ ]:
doc=chroma_db.as_retriever().get_relevant_documents(query)

In [ ]:
retrieval_grader=grade_prompt|structure_llm_grader

In [ ]:
retrieval_grader.invoke({"question":query,"documents":doc[0].page_content})

In [ ]:
retrieval_grader.get_graph()